<a href="https://colab.research.google.com/github/boramkim0514/KOIPA_AI/blob/main/D57_58_customYOLO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
!ls -al '/gdrive/MyDrive/open_cv/darknet'

total 256007
drwx------ 2 root root      4096 Jul 21 03:50 backup
drwx------ 2 root root      4096 Jul 21 03:50 bin
drwx------ 2 root root      4096 Jul 21 03:50 cfg
drwx------ 2 root root      4096 Jul 21 03:50 custom
drwx------ 2 root root      4096 Jul 21 03:50 data
-rw------- 1 root root   2670159 Jul 22 07:26 fruit10.jpg
drwx------ 2 root root      4096 Jul 21 03:50 images
drwx------ 2 root root      4096 Jul 22 05:47 my_yolo
-rw------- 1 root root    828075 Jul 22 07:31 predictions.jpg
drwx------ 2 root root      4096 Jul 21 03:50 trafficsign
-rw------- 1 root root 258615009 Mar 18 02:18 Train.zip
drwx------ 2 root root      4096 Jul 21 03:50 weights


In [ ]:
COLAB_DARKNET_PATH = '/gdrive/MyDrive/open_cv/darknet'
DATA_PATH = COLAB_DARKNET_PATH+"/my_yolo/data"

YOLO_IMAGE_PATH = DATA_PATH
YOLO_FORMAT_PATH = DATA_PATH

## Create Config Files


In [ ]:
class_count = 0
test_ratio = 0.2
paths = []

#1. clasees.names 파일 
*클래스명이 써져있는 파일

In [ ]:
# !fusermount -u drive

# !google-drive-ocamlfuse drive

In [ ]:
with open(YOLO_FORMAT_PATH+"/classes.names","w") as names, open(YOLO_FORMAT_PATH+"/classes.txt") as txt:
    for line in txt:
        names.write(line)
        class_count += 1
    print("[classes.names] is created")

[classes.names] is created


#2. `.data` 파일 만들기
* 파일의 이름은 마음대로 저장 가능!
* darknet에서는 이 파일을 이용해서 어디서 훈련 및 검증할 데이터를 가져올지, 클래스는 몇개인지, 백업은 어디다 할지 등등을 기록

> Indented block



In [ ]:
with open(YOLO_FORMAT_PATH + "/" + "custom_data.data", "w") as data:
    data.write("classes = " + str(class_count) + "\n")
    data.write("train = " + YOLO_FORMAT_PATH + "/" +"train.txt" + "\n")
    data.write("valid = " + YOLO_FORMAT_PATH + "/" +"test.txt" + "\n")
    data.write("names = " + YOLO_FORMAT_PATH + "/" +"classes.names" + "\n")
    data.write("bacup = backup")
    print("[custum_data.data] is created")

[custum_data.data] is created


#3. train/test 데이터 지정하기 

In [ ]:
import os
for current_dir, dirs, files, in os.walk(YOLO_IMAGE_PATH):
    # print(current_dir, dirs, files)
    for file in files:
        if file.endswith(".jpg"):
            image_path = YOLO_IMAGE_PATH + "/" + file
            print(image_path)
            
            paths.append(image_path+"\n")

/gdrive/MyDrive/open_cv/darknet/my_yolo/data/fruit05.jpg
/gdrive/MyDrive/open_cv/darknet/my_yolo/data/fruit06.jpg
/gdrive/MyDrive/open_cv/darknet/my_yolo/data/fruit01.jpg
/gdrive/MyDrive/open_cv/darknet/my_yolo/data/fruit09.jpg
/gdrive/MyDrive/open_cv/darknet/my_yolo/data/fruit04.jpg
/gdrive/MyDrive/open_cv/darknet/my_yolo/data/fruit08.jpg
/gdrive/MyDrive/open_cv/darknet/my_yolo/data/fruit02.jpg
/gdrive/MyDrive/open_cv/darknet/my_yolo/data/fruit07.jpg
/gdrive/MyDrive/open_cv/darknet/my_yolo/data/fruit03.jpg
/gdrive/MyDrive/open_cv/darknet/my_yolo/data/fruit10.jpg


In [ ]:
paths

['/gdrive/MyDrive/open_cv/darknet/my_yolo/data/fruit05.jpg\n',
 '/gdrive/MyDrive/open_cv/darknet/my_yolo/data/fruit06.jpg\n',
 '/gdrive/MyDrive/open_cv/darknet/my_yolo/data/fruit01.jpg\n',
 '/gdrive/MyDrive/open_cv/darknet/my_yolo/data/fruit09.jpg\n',
 '/gdrive/MyDrive/open_cv/darknet/my_yolo/data/fruit04.jpg\n',
 '/gdrive/MyDrive/open_cv/darknet/my_yolo/data/fruit08.jpg\n',
 '/gdrive/MyDrive/open_cv/darknet/my_yolo/data/fruit02.jpg\n',
 '/gdrive/MyDrive/open_cv/darknet/my_yolo/data/fruit07.jpg\n',
 '/gdrive/MyDrive/open_cv/darknet/my_yolo/data/fruit03.jpg\n',
 '/gdrive/MyDrive/open_cv/darknet/my_yolo/data/fruit10.jpg\n']

In [ ]:
num_test = int(len(paths) * test_ratio)

#train/ test 분리
paths_test = paths[:num_test]
paths_train = paths[num_test:]

with open(YOLO_FORMAT_PATH + "/" + "train.txt", "w") as train_txt:
    for path in paths_train:
        train_txt.write(path)
    print("[train.txt] is created")

[train.txt] is created


In [ ]:
# text.txt
with open(YOLO_FORMAT_PATH + "/" + "test.txt", "w") as test_txt:
  for path in paths_test:
    test_txt.write(path)
  print("[test.txt] is created")

[test.txt] is created


### cuda 버전 확인

In [ ]:
!/usr/local/cuda/bin/nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Wed_Jul_22_19:09:09_PDT_2020
Cuda compilation tools, release 11.0, V11.0.221
Build cuda_11.0_bu.TC445_37.28845127_0


### os 확인하기

In [ ]:
import platform
platform.platform()

'Linux-5.4.104+-x86_64-with-Ubuntu-18.04-bionic'

# Default Darknet YOLO.v3 사용하기 런타임 GPU로 변경

In [ ]:
!wget https://pjreddie.com/media/files/yolov3.weights



--2021-07-23 00:14:25--  https://pjreddie.com/media/files/yolov3.weights
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248007048 (237M) [application/octet-stream]
Saving to: ‘yolov3.weights’

yolov3.weights      100%[===================>] 236.52M  91.9MB/s    in 2.6s    

2021-07-23 00:14:28 (91.9 MB/s) - ‘yolov3.weights’ saved [248007048/248007048]



In [ ]:
#darknet 실행 파일 확인
!ls -al "/gdrive/MyDrive/open_cv/darknet/bin"

total 3139
-rw------- 1 root root 3213624 Apr 19  2020 darknet


In [ ]:
# 실행 위치 변경
os.chdir("/gdrive/MyDrive/open_cv/darknet")

In [ ]:
print(os.path.abspath(os.curdir))

/gdrive/MyDrive/open_cv/darknet


In [ ]:
!ls -al

total 256007
drwx------ 2 root root      4096 Jul 21 03:50 backup
drwx------ 2 root root      4096 Jul 21 03:50 bin
drwx------ 2 root root      4096 Jul 21 03:50 cfg
drwx------ 2 root root      4096 Jul 21 03:50 custom
drwx------ 2 root root      4096 Jul 21 03:50 data
-rw------- 1 root root   2670159 Jul 22 07:26 fruit10.jpg
drwx------ 2 root root      4096 Jul 21 03:50 images
drwx------ 3 root root      4096 Jul 22 05:47 my_yolo
-rw------- 1 root root    828075 Jul 22 07:31 predictions.jpg
drwx------ 2 root root      4096 Jul 21 03:50 trafficsign
-rw------- 1 root root 258615009 Mar 18 02:18 Train.zip
drwx------ 2 root root      4096 Jul 21 03:50 weights


In [ ]:
# 다크넷 파일 실행 권한 변경
!chmod +x ./bin/darknet

In [ ]:
from google.colab import files
files.upload()

KeyboardInterrupt: ignored

In [ ]:
!./bin/darknet detect cfg/yolov3.cfg weights/yolov3.weights fruit10.jpg

layer     filters    size              input                output
   0 conv     32  3 x 3 / 1   416 x 416 x   3   ->   416 x 416 x  32 0.299 BF
   1 conv     64  3 x 3 / 2   416 x 416 x  32   ->   208 x 208 x  64 1.595 BF
   2 conv     32  1 x 1 / 1   208 x 208 x  64   ->   208 x 208 x  32 0.177 BF
   3 conv     64  3 x 3 / 1   208 x 208 x  32   ->   208 x 208 x  64 1.595 BF
   4 Shortcut Layer: 1
   5 conv    128  3 x 3 / 2   208 x 208 x  64   ->   104 x 104 x 128 1.595 BF
   6 conv     64  1 x 1 / 1   104 x 104 x 128   ->   104 x 104 x  64 0.177 BF
   7 conv    128  3 x 3 / 1   104 x 104 x  64   ->   104 x 104 x 128 1.595 BF
   8 Shortcut Layer: 5
   9 conv     64  1 x 1 / 1   104 x 104 x 128   ->   104 x 104 x  64 0.177 BF
  10 conv    128  3 x 3 / 1   104 x 104 x  64   ->   104 x 104 x 128 1.595 BF
  11 Shortcut Layer: 8
  12 conv    256  3 x 3 / 2   104 x 104 x 128   ->    52 x  52 x 256 1.595 BF
  13 conv    128  1 x 1 / 1    52 x  52 x 256   ->    52 x  52 x 128 0.177 BF
  14 c

In [ ]:
import cv2
from google.colab.patches import cv2_imshow

img = cv2.imread("predictions.jpg")
cv2_imshow(img)

## Custom YOLO

In [ ]:
!ls -al

# 백업 폴더 만들기(backup=backup) 안 만들면 에러!
!mkdir /bin/backup

In [ ]:
!bin/darknet detector train my_yolo/data/custom_data.data my_yolo/cfg/custom-train-yolo.cfg weights/darknet53.conv.74 -dont_show

In [ ]:
!bin/darknet detector map my_yolo/data/custom_data.data my_yolo/cfg/custom-train-yolo.cfg backup/custom-train-yolo_final.weights -dont_show

In [ ]:
!bin/darknet detector test my_yolo/data/custom_data.data my_yolo/cfg/custom-train-yolo.cfg backup/custom-train-yolo_final.weights fruit10.jpg -dont_show

In [ ]:
img = cv2.imread("predictions.jpg")
cv2_imshow(img)